In [1]:
import pandas as pd
import requests
from pathlib import Path
import subprocess
import os
from google.cloud import storage
from google.cloud.storage import transfer_manager
import re
import pickle
import numpy as np
import multiprocessing

In [2]:
df = pd.read_csv("data/pifsc_products_detections_annotations.csv")

# Fixes bug with hawaii 
df['hawaii_num'] = df[df['flac_compressed_xwav_object'].str.contains("pipan_hawaii_0/")]['flac_compressed_xwav_object'].str.split("/", expand=True).iloc[:, -1].str.split("_", expand=True).loc[:, 2].astype(float)

df.loc[:, "flac_compressed_xwav_object"] = df.apply(lambda x: x['flac_compressed_xwav_object'].replace("pipan_hawaii_0/", f"pipan_hawaii_{int(x['hawaii_num'])}/") if not np.isnan(x["hawaii_num"]) else x["flac_compressed_xwav_object"], axis=1)

df.head()

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc,hawaii_num
0,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1,True,True,Other,0.213,0.213,2005-06-06 13:00:00.213000+00:00,2005-06-06 13:00:00.213000+00:00,NaN
1,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,13,True,True,Other,0.108,0.108,2005-06-06 14:15:00.108000+00:00,2005-06-06 14:15:00.108000+00:00,NaN
2,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Other,0.143,0.143,2005-07-02 04:00:00.143000+00:00,2005-07-02 04:00:00.143000+00:00,NaN
3,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,17,True,True,Other,0.213,0.213,2005-07-02 05:15:00.213000+00:00,2005-07-02 05:15:00.213000+00:00,NaN
4,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,22,True,True,Other,0.213,0.213,2005-08-01 10:15:00.213000+00:00,2005-08-01 10:15:00.213000+00:00,NaN


In [3]:
with open("file_manifest.txt", "r") as f:
    processed_files = f.readlines()
processed_files = [f.strip().split(".")[0] for f in processed_files]
processed_files[:5]

['Cross_A_01_050514_014345',
 'Cross_A_01_050518_040230',
 'Cross_A_01_050520_183000',
 'Cross_A_01_050520_212731',
 'Cross_A_01_050521_004500']

In [4]:
files = list(df['flac_compressed_xwav_object'].unique())
files = [re.sub("gs://noaa-passive-bioacoustic/", "", f) for f in files]
print(len(files))
new_files = []

for file in files:
    hold = False
    for processed_file in processed_files:
        if processed_file in file:
            hold = True
            break
    if not hold:
        new_files.append(file)
    
print(len(new_files), len(processed_files))
files = new_files
# files = files[0:1000]

def download_many_blobs_with_transfer_manager(bucket_name, blob_names, destination_directory="", workers=8):
    storage_client = storage.Client.create_anonymous_client()
    
    bucket = storage_client.bucket(bucket_name)
    
    results = transfer_manager.download_many_to_path(bucket, blob_names, destination_directory=destination_directory, max_workers=workers)
    
    bad_results = []
    for name, result in zip(blob_names, results):
        if isinstance(result, Exception):
            bad_results.append((name, result))
            print(f"Failed to download {name} due to exception {result}")
        else:
            print(f"Download of {name} succeeded")

    return bad_results
files[:5], len(files)

5294
4282 1013


(['pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120204_054115.df20.x.flac',
  'pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120206_022345.df20.x.flac',
  'pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120207_004500.df20.x.flac',
  'pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120207_230615.df20.x.flac',
  'pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120208_080729.df20.x.flac'],
 4282)

In [7]:
downloaded_files = []
for dirpath, dirnames, filenames in os.walk("D:/SonarData/downloads"):
    for filename in filenames:
        downloaded_files.append(filename)
        
to_download = [f for f in files if f.split("/")[-1] not in downloaded_files]
to_download[:5], len(to_download)

(['pifsc/audio/pipan/saipan/pipan_saipan_06/audio/Saipan_A_06_160225_053445.df20.x.flac',
  'pifsc/audio/pipan/saipan/pipan_saipan_06/audio/Saipan_A_06_160226_130100.df20.x.flac',
  'pifsc/audio/pipan/saipan/pipan_saipan_06/audio/Saipan_A_06_160227_202515.df20.x.flac',
  'pifsc/audio/pipan/saipan/pipan_saipan_06/audio/Saipan_A_06_160229_034229.df20.x.flac',
  'pifsc/audio/pipan/saipan/pipan_saipan_06/audio/Saipan_A_06_160301_105945.df20.x.flac'],
 3782)

In [6]:
bad_results = download_many_blobs_with_transfer_manager("noaa-passive-bioacoustic", 
                                                        to_download[:500],
                                                        destination_directory="D:/SonarData/downloads",
                                                        workers=8)

Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120204_054115.df20.x.flac succeeded
Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120206_022345.df20.x.flac succeeded
Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120207_004500.df20.x.flac succeeded
Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120207_230615.df20.x.flac succeeded
Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120208_080729.df20.x.flac succeeded
Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120209_062845.df20.x.flac succeeded
Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120211_031115.df20.x.flac succeeded
Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120212_013230.df20.x.flac succeeded
Download of pifsc/audio/pipan/kingman/pipan_kingman_01/audio/Kingman_A_01_120212_235345.df20.x.flac succeeded
Download o

In [17]:
len(bad_results)

0

In [11]:
with open("bad_results.pkl", 'wb') as f:
    pickle.dump(bad_results, f)

In [78]:
with open("bad_results.pkl", 'rb') as f:
    bad_results = pickle.load(f)
bad_files = [f[0] for f in bad_results]
bad_file_names = {f.split("/")[-1] for f in bad_files}
list(bad_file_names)[:5]

['Hawaii_K_16_140207_060115.df20.x.flac',
 'Hawaii_K_16_140105_233000.df20.x.flac',
 'Hawaii_K_11_110924_072800.df20.x.flac',
 'Hawaii_K_14_121222_090345.df20.x.flac',
 'Hawaii_K_14_121228_162345.df20.x.flac']

In [79]:
for dirpath, dirnames, filenames in os.walk("D:/SonarData/downloads"):
    for filename in filenames:
        if filename in bad_file_names:
            os.remove(os.path.join(dirpath, filename))

In [10]:
files = os.listdir("D:/SonarData/flat")
files[:10]

['Kingman_A_01_120204_054115.df20.x.flac',
 'Kingman_A_01_120206_022345.df20.x.flac',
 'Kingman_A_01_120207_004500.df20.x.flac',
 'Kingman_A_01_120207_230615.df20.x.flac',
 'Kingman_A_01_120208_080729.df20.x.flac',
 'Kingman_A_01_120209_062845.df20.x.flac',
 'Kingman_A_01_120211_031115.df20.x.flac',
 'Kingman_A_01_120212_013230.df20.x.flac',
 'Kingman_A_01_120212_235345.df20.x.flac',
 'Kingman_A_01_120213_221500.df20.x.flac']

In [ ]:
def uncompress(files_to_decompress):
    for i, file in enumerate(files_to_decompress):
        print(file)
        subprocess.run(f"C:/Windows/flac.exe -df --delete-input-file --preserve-modtime --keep-foreign-metadata D:/SonarData/messing/{file}", shell=True, capture_output=True)
    return True

pool = multiprocessing.Pool(6)
messing_files = os.listdir("D:/SonarData/messing")
step = int(len(messing_files)/6)
hold = [messing_files[j: j + step] for j in [i for i in range(0, len(messing_files), step)]]

r = pool.map_async(uncompress, hold)
r.wait()

In [21]:
hold

[['Kingman_A_01_120204_054115.df20.x.flac',
  'Kingman_A_01_120206_022345.df20.x.flac',
  'Kingman_A_01_120207_004500.df20.x.flac'],
 ['Kingman_A_01_120207_230615.df20.x.flac',
  'Kingman_A_01_120208_080729.df20.x.flac',
  'Kingman_A_01_120209_062845.df20.x.flac'],
 ['Kingman_A_01_120211_031115.df20.x.flac',
  'Kingman_A_01_120212_013230.df20.x.flac',
  'Kingman_A_01_120212_235345.df20.x.flac'],
 ['Kingman_A_01_120213_221500.df20.x.flac',
  'Kingman_A_01_120214_203615.df20.x.flac',
  'Kingman_A_01_120215_185730.df20.x.flac'],
 ['Kingman_A_01_120218_140115.df20.x.flac',
  'Kingman_A_01_120219_122230.df20.x.flac',
  'Kingman_A_01_120220_104345.df20.x.flac'],
 ['Kingman_A_01_120221_090500.df20.x.flac',
  'Kingman_A_01_120222_072615.df20.x.flac',
  'Kingman_A_01_120223_054730.df20.x.flac'],
 ['Kingman_A_01_120224_040845.df20.x.flac',
  'Kingman_A_01_120225_023000.df20.x.flac']]

In [4]:
# Uncompress data
files = os.listdir("D:/SonarData/flat")
new_files = [i for i in files if i.endswith(".flac")]
for i, file in enumerate(new_files):
    if i % 50 == 0:
        print(f"Processing file {i}")
    subprocess.run(f"C:/Windows/flac.exe -df --delete-input-file --preserve-modtime --keep-foreign-metadata D:/SonarData/flat/{file}", shell=True, capture_output=True)

Processing file 0
Processing file 50
Processing file 100
Processing file 150
Processing file 200
Processing file 250
Processing file 300
Processing file 350
Processing file 400
Processing file 450


In [12]:
new_files

['Cross_A_01_050514_014345.d20.x.flac',
 'Cross_A_01_050518_040230.d20.x.flac',
 'Cross_A_01_050520_183000.d20.x.flac',
 'Cross_A_01_050520_212731.d20.x.flac',
 'Cross_A_01_050521_004500.d20.x.flac',
 'Cross_A_01_050527_003345.d20.x.flac',
 'Cross_A_01_050606_123845.d20.x.flac',
 'Cross_A_01_050612_151845.d20.x.flac',
 'Cross_A_01_050613_232350.d20.x.flac',
 'Cross_A_01_050630_004730.d20.x.flac',
 'Cross_A_01_050630_222845.d20.x.flac',
 'Cross_A_01_050701_112115.d20.x.flac',
 'Cross_A_01_050702_031345.d20.x.flac',
 'Cross_A_01_050703_170730.d20.x.flac',
 'Cross_A_01_050717_151615.d20.x.flac',
 'Cross_A_01_050722_115500.d20.x.flac',
 'Cross_A_01_050801_074615.d20.x.flac',
 'Cross_A_01_050801_110500.d20.x.flac',
 'Cross_A_01_050908_025000.d20.x.flac',
 'Cross_A_02_051120_000000.d20.x.flac']

In [8]:
len(files)

1012

In [15]:
files = os.listdir("D:/SonarData/flat")
len([i for i in files if i.endswith(".flac")])

992